<div style="display: flex; align-items: flex-start;">
  <div style="flex: 1; max-width: 20%; padding-right: 10px;">
    <img src="./images/ChatLLM-logo.png" width=250px>
  </div>
  <div style="flex: 2; max-width: 80%;">
    <center><h1>ChatLLM</h1></center>
    <p>ChatLLM is a versatile conversational platform designed to streamline interactions with <u>multiple language models</u>. <br><br> It allows users to <b>switch seamlessly between various models</b>, such as OpenAI's GPT series, LLaMA, and Mistral, within a single conversation. This <u>eliminates the need to juggle multiple browser tabs</u>, enabling efficient exploration of each model's unique capabilities in one cohesive interface. <br><br> With ChatLLM, users can maintain a unified chat history while dynamically selecting the best model for their needs during the conversation.</p>
  </div>
</div>

# Imports

Import all the necessary libraries to launch ChatLLM

In [1]:
import os
from dotenv import load_dotenv
import json
import requests
import subprocess

import gradio as gr
import openai
from openai import OpenAI
import ollama

# Environment Variables and API Validation

Starts loading environment variables from a `.env` file inside the project folder

Create the `.env` file with the following cell, open it and store inside it your API keys:
<br>
<br>➡️ OPENAI_API_KEY has to be setted with your OpenAI API key created from https://platform.openai.com/account
<br>➡️ OLLAMA_API_KEY is already setted

In [1]:
! [ ! -f .env ] && touch .env && echo "OPENAI_API_KEY=your_openai_api_key_here"> .env && echo "OLLAMA_API_KEY=http://localhost:11434/">> .env

Load the environment variables from the `.env` file

In [3]:
print("⏳ Loading environment variables...")

# load environment variables
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
ollama_api_key = os.getenv('OLLAMA_API_KEY')

# check OpenAI API key
if not openai_api_key or not openai_api_key.startswith("sk-proj-"):
    print("❌ ERROR: OpenAI API key invalid or missing.")
else:
    print("✅ OpenAI API key found and valid.")

# check Ollama server
try:
    response = requests.get(ollama_api_key, timeout=5)
    if response.status_code == 200:
        print("✅ Ollama server is already running.")
except requests.exceptions.RequestException:
    print("❌ ERROR: Ollama server is not running. Attempting to start it...")

    try:
        subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("⏳ Starting Ollama server... Waiting for it to be reachable.")
        
        for _ in range(100):
            try:
                response = requests.get(ollama_api_key, timeout=5)
                if response.status_code == 200:
                    print("✅ Ollama server started successfully.")
                    break
                    
            except requests.exceptions.RequestException:
                pass
        
    except FileNotFoundError:
        print("❌ ERROR: Ollama command not found. Download Ollama from: https://ollama.com/download.")
        
    except Exception as e:
        print(f"❌ ERROR: Failed to start Ollama server. Exception: {e}.")
    

⏳ Loading environment variables...
✅ OpenAI API key found and valid.
❌ ERROR: Ollama server is not running. Attempting to start it...
⏳ Starting Ollama server... Waiting for it to be reachable.


# Choose your system prompt

Write down all the necessary information your AI assistant should need to help you in your task. Write it inside a `.txt` file inside the folder `prompts`

In [4]:
PROMPT_PATH = 'prompts'
my_system_prompt = 'system_prompt.txt'

# read the prompt file
file_path=os.path.join(PROMPT_PATH, my_system_prompt)

try:
    with open(file_path, 'r') as file:
        system_message = file.read()
except FileNotFoundError:
    print(f"❌ ERROR: The file at {file_path} was not found.")
except IOError as e:
    print(f"❌ ERROR: An error occurred while reading the file: {e}")
else:
    print(f"✅ System Prompt read successfully: {system_message}")

✅ System Prompt read successfully: You are a strict code reviewer who focuses on identifying inefficiencies and potential bugs in Python code.


# Functions

Imports necessary functions to build a message in the OpenAI format, call the different models and utility functions

In [5]:
def build_messages(message, history):
    
    # convert Gradio history to OpenAI message format
    messages = [
        {
            "role": "system",
            "content": system_message
        }
    ]
    
    # add message pairs from history
    for user_msg, assistant_msg in history:
        messages.append(
            {
                "role": "user",
                "content": user_msg
            }
        )
        messages.append(
            {
                "role": "assistant",
                "content": assistant_msg
            }
        )
    
    # add the current message
    messages.append(
        {
            "role": "user",
            "content": message
        }
    )

    return messages

def chat(message, history, model, debug=False):
    
    if 'gpt' in model:
        
        # build messages
        messages = build_messages(message, history)
        
        if debug:
            print("History is:")
            print(json.dumps(history, indent=4))
            print("And messages is:")
            print(json.dumps(messages, indent=4))
        
        # call GPT
        stream = openai.chat.completions.create(
            model=model, 
            messages=messages, 
            stream=True
        )
        
        # return response
        response = ""
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            yield history + [(message, response)]

    elif 'llama' in model or 'mistral' in model or 'qwen' in model or 'deepseek' in model:
        
        # check model
        try:
            models = ollama.list()
            model_names = [m.model.split(':')[0] for m in models.models]
            
            if model not in model_names:
                print(f"🚨 Model {model} not found. Downloading...")
                ollama.pull(model)
                print(f"✅ Model {model} downloaded successfully")

        except Exception as e:
            raise Exception(f"❌ ERROR: checking/downloading model lead to {e}")

        # build messages
        messages = build_messages(message, history)
        
        if debug:
            print("History is:")
            print(json.dumps(history, indent=4))
            print("And messages is:")
            print(json.dumps(messages, indent=4))
        
        # call Ollama
        stream = ollama.chat(
            model=model,
            messages=messages,
            stream=True
        )
        
        # return response
        response = ""
        for chunk in stream:
            response += chunk.get('message', {}).get('content', '')
            yield history + [(message, response)]

def clear_input():
        return gr.Textbox(value="")


# Call ChatLLM!

Call the model interface

In [6]:
# build interface
with gr.Blocks() as demo:

    gr.Markdown("## ChatLLM! The right model at the right time")
    chatbot = gr.Chatbot(label=None, show_label=False, placeholder="<center><h1><bold>Welcome to ChatLLM</bold></h1>How can I help you?</center>")
    
    with gr.Row():
        msg = gr.Textbox(label=None, show_label=False, placeholder="Enter your message to ChatLLM", scale=7)
        model = gr.Dropdown(
            ["gpt-4o-mini", "gpt-4o", "gpt-4", "gpt-3.5-turbo", "llama3.2", "llama3.1", "mistral", "qwen2.5", "qwen2.5-coder", "deepseek-r1"],
            label=None,
            show_label=False,
            value="llama3.2",
            scale=2
        )

    msg.submit(
        chat,
        inputs=[msg, chatbot, model],
        outputs=chatbot
    )
    
    msg.submit(
        clear_input,
        inputs=[],
        outputs=[msg]
    )

demo.launch(
    share=False,
    inbrowser=True
)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
